# Loading and Analysing Pre-Trained Sparse Autoencoders

## Imports & Installs

## Set Up

In [ ]:
try:
    import google.colab # type: ignore
    from google.colab import output
    COLAB = True
    %pip install sae-lens transformer-lens
except:
    COLAB = False
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# Standard imports
import os
import plotly.express as px
import pandas as pd
import json
import numpy as np
import math
import pandas as pd
import random
import shutil
import networkx as nx
import ast

from collections import Counter
from functools import partial
from tqdm import tqdm
from faker import Faker

import torch
torch.set_grad_enabled(False);
from openai import AzureOpenAI
from datasets import load_dataset  
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

import transformer_lens
from transformer_lens import utils
from transformer_lens import HookedTransformer
from transformer_lens.utils import tokenize_and_concatenate

from sae_lens import SAE
from sae_lens.config import DTYPE_MAP, LOCAL_SAE_MODEL_PATH
from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list

from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData

from causallearn.search.ConstraintBased.FCI import fci
from causallearn.search.ConstraintBased.PC import pc
from causallearn.search.ScoreBased.ExactSearch import bic_exact_search
from causallearn.utils.cit import kci
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.utils.PCUtils.BackgroundKnowledge import BackgroundKnowledge
from causallearn.utils.cit import fisherz

In [ ]:
# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

# Loading a pretrained Sparse Autoencoder

Below we load a Transformerlens model, a pretrained SAE and a dataset from huggingface.

In [ ]:
NUM_PLAYERS_GENERATE = 25 ###
NUM_PLAYERS_USE = 25 ###
NUM_PLAYERS_START = -1 ###

NUM_VALUE_DIM = 'SMALLSET'#'ALL', 'SMALLSET', 100
MAX_QUESTIONS_PER_BATCH = 8
GENERATE_NEW_PLAYERS = False

PERSON = 0 ###V for case test, 0 for main, 2 for second person, 3 for third person, 4 for Inversion Value Def, 5 for Value Def
ALLOW_UNSURE_ANSWER = False
SYSTEMATIC_PROMPT = 1 ###LLAMA
EXAMPLES_IN_PROMPT = 1

SAE_STEERED_RANGE = 'onlyvalue' #'roleinstruction','onlyvalue' 
SAE_STEERED_FEATURE_NUM = 25 ###25, 10
SAE_STEERED_FEATURE_BAN = []
#SAE_STEERED_FEATURE_BAN = [10096, 8387, 2221, 1312, 7502, 14049]
#SAE_STEERED_FEATURE_BAN = [60312, 7754, 13033]
#SAE_STEERED_FEATURE_BAN = [60312, 7754, 13033, 1897, 2509, 20141, 41929, 48321, 63905]
#SAE_STEERED_FEATURE_BAN = [60312, 7754, 13033, 1897, 2509, 20141, 41929, 48321, 63905, 49202, 2246, 58305]

SAMPLING_KWARGS = dict(max_new_tokens=100, do_sample=False, temperature=0.5, top_p=0.7, freq_penalty=1.0, ) ##LLAMA
STEERING_ON = True
STEER_LOC = 'out' # 'in', 'out'
STEER_COEFF = 100

JUDGE_ANSWER_RULE_FIRST = False
JUDGE_ANSWER_WITH_YESNO = False

VERBOSE = False

GROUP_SAMPLE_RATE = 1 ###0.4 for valuebenchtrain
DATA_SPLIT = 'valuebenchtest' ###
if DATA_SPLIT == '30clearori':
    df_valuebench = pd.read_csv(os.path.join(LOCAL_SAE_MODEL_PATH, 'value_data/value_orientation_30clearori.csv'))
elif DATA_SPLIT == 'other':
    df_valuebench = pd.read_csv(os.path.join(LOCAL_SAE_MODEL_PATH, 'value_data/value_orientation_other.csv'))
elif DATA_SPLIT == 'valuebenchtrain':
    df_valuebench = pd.read_csv(os.path.join(LOCAL_SAE_MODEL_PATH, 'value_data/value_orientation_train.csv'))
elif DATA_SPLIT == 'valuebenchtest':
    df_valuebench = pd.read_csv(os.path.join(LOCAL_SAE_MODEL_PATH, 'value_data/value_orientation_test.csv'))
elif DATA_SPLIT == 'valuebenchteaser':
    df_valuebench = pd.read_csv(os.path.join(LOCAL_SAE_MODEL_PATH, 'value_data/value_orientation_teaser.csv'))
elif DATA_SPLIT == 'valuebenchall':
    df_valuebench = pd.read_csv(os.path.join(LOCAL_SAE_MODEL_PATH, 'value_data/value_orientation.csv'))    
else:
    raise ValueError('Invalid data split')


grouped = df_valuebench.groupby('value')
if NUM_VALUE_DIM != 'ALL':
    if NUM_VALUE_DIM == 'TEASERSET':
        smallset = ["Uncertainty Avoidance", "Breadth of Interest", "Social Cynicism"] #values with 20+ questions in valuebench
        grouped = [group for group in grouped if group[0] in smallset]
    elif NUM_VALUE_DIM == 'SMALLSET':
        #smallset = ['Indulgence', 'Hedonism']
        #smallset = ['Laziness', 'Workaholism']
        #smallset = ['Achievement']
        #smallset = ['Empathy', 'Sympathy']
        #smallset = ['Assertiveness']
        #smallset = ['Preference for Order and Structure']
        #smallset = ['Assertiveness', 'Breadth of Interest', 'Empathy', 'Extraversion', 'Nurturance', 'Preference for Order and Structure', 'Sociability', 'Social', 'Sympathy', 'Tenderness', 'Theoretical', 'Understanding'] #values with 9+ questions in 30clearori
        smallset = ["Positive coping", "Empathy", "Resilience", "Social Complexity", "Achievement", "Uncertainty Avoidance", "Aesthetic", "Anxiety Disorder", "Breadth of Interest", "Economic", "Organization", "Political", "Religious", "Social", "Social Cynicism", "Theoretical", "Understanding"] #values with 20+ questions in valuebench
        #smallset = ["Sociability", "Perfectionism", "Assertiveness", "Creativity", "Emotional expressiveness", "Religiosity", "Reward for Application", "Anxiety", "Dominance", "Conscientiousness", "Preference for Order and Structure", "Rationality", "Discomfort with Ambiguity", "Dutifulness", "Independence", "Individualism", "Nurturance", "Preference for Predictability", "Sympathy", "Tenderness"] #values with 13-19 questions in valuebench
        #smallset = ['Social', 'Understanding', 'Empathy', 'Breadth of Interest', 'Theoretical']#intersection of values with 9+ questions in 30clearori and 20+ questions in valuebench
        grouped = [group for group in grouped if group[0] in smallset]
    else:
        grouped = random.sample(list(grouped), NUM_VALUE_DIM)

#prepapre value description and examples
value_list = [group[0] for group in grouped]
value_def_file = os.path.join(LOCAL_SAE_MODEL_PATH, 'value_data/definitions.csv')
value_def_data = pd.read_csv(value_def_file)
value_def_dict = {}
for value_name in value_list:
    value_def_dict[value_name] = ' '.join([vd for vd in value_def_data[value_def_data['value'] == value_name]['definition'].values if isinstance(vd, str)])
    

if not os.path.exists('valuebench_info'):
    os.mkdir('valuebench_info')
else:
    shutil.rmtree('valuebench_info')
    os.mkdir('valuebench_info')

for value_name, value_qa in grouped:
    print(value_name, len(value_qa))
    with open(os.path.join('valuebench_info','value_questions_' + value_name + '.html'), 'w') as f:
        for question, answer in zip(value_qa['question'], value_qa['agreement']):
            f.write(f'<p>Question: {question}</p>')
            f.write(f'<p>Postive Answer: {answer}</p>')
            f.write(f'<p>==============================</p>')

GPT_client = AzureOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.environ.get("OPENAI_BASE_URL"),
    #base_url=os.environ.get("OPENAI_BASE_URL"),
)

base_model = 'GEMMA-2B-IT'
#base_model = 'GEMMA-2B-CHN'
#base_model = 'GPT2-SMALL'
#base_model = 'MISTRAL-7B'
#base_model = 'LLAMA3-8B'
#base_model = 'LLAMA3-8B-IT'
#base_model = 'LLAMA3-8B-IT-HELPFUL'
#base_model = 'LLAMA3-8B-IT-CHN'
#base_model = 'LLAMA3-8B-IT-FICTION'
#base_model = 'MISTRAL-7B'


if base_model == 'LLAMA3-8B-IT':
    STOP_SIGNS = [128001,128009] ##LLAMA
else:
    STOP_SIGNS = None



if base_model == 'GPT2-SMALL':
    answer_valuebench_features_csv = 'answers_gpt2small' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'GEMMA-2B-IT':
    answer_valuebench_features_csv = 'answers_gemma2bit' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'GEMMA-2B':
    answer_valuebench_features_csv = 'answers_gemma2b' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'GEMMA-2B-CHN':
    answer_valuebench_features_csv = 'answers_gemma2bchn' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'MISTRAL-7B':
    answer_valuebench_features_csv = 'answers_mistral7b' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'LLAMA3-8B':
    answer_valuebench_features_csv = 'answers_llama38b' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'LLAMA3-8B-IT':
    answer_valuebench_features_csv = 'answers_llama38bit' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'LLAMA31-8B-IT':
    answer_valuebench_features_csv = 'answers_llama318bit' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'LLAMA3-8B-IT-HELPFUL':
    answer_valuebench_features_csv = 'answers_llama38bithelp' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'LLAMA3-8B-IT-FICTION':
    answer_valuebench_features_csv = 'answers_llama38bitfiction' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
elif base_model == 'LLAMA3-8B-IT-CHN':
    answer_valuebench_features_csv = 'answers_llama38bitchn' + '_players'+ str(NUM_PLAYERS_GENERATE) + '_valuedims' + str(NUM_VALUE_DIM) +'.csv'
else:
    raise ValueError('Invalid base model')

if ALLOW_UNSURE_ANSWER:
    answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_unsure.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_sae.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'NUM_PLAYERS_USE' + str(NUM_PLAYERS_USE) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'NUM_PLAYERS_START' + str(NUM_PLAYERS_START) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'SAE_STEERED_FEATURE_NUM' + str(SAE_STEERED_FEATURE_NUM) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'PERSON' + str(PERSON) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'SYSTEMATIC_PROMPT' + str(SYSTEMATIC_PROMPT) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'EXAMPLES_IN_PROMPT' + str(EXAMPLES_IN_PROMPT) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'SAE_STEERED_RANGE' + str(SAE_STEERED_RANGE) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'STEERING_COEFF' + str(STEER_COEFF) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + str(DATA_SPLIT) + '.csv')
answer_valuebench_features_csv = answer_valuebench_features_csv.replace('.csv', '_' + 'SAMPLERATE' + str(GROUP_SAMPLE_RATE) + '.csv')

answer_valuebench_features_csv = os.path.join('useful_data', answer_valuebench_features_csv)
# JUDGE_ANSWER_RULE_FIRST
# JUDGE_ANSWER_WITH_YESNO

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )

In [ ]:
if base_model == 'GPT2-SMALL':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "openai-community", "gpt2"))
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "openai-community", "gpt2"), padding_side='left')

elif base_model == 'GEMMA-2B':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "google", "gemma-2b"))
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "google", "gemma-2b"), padding_side='left')

elif base_model == 'GEMMA-2B-IT':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "google", "gemma-2b-it"))
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "google", "gemma-2b-it"), padding_side='left')
    #vllm_generator = VLLMGenerator(os.path.join(LOCAL_SAE_MODEL_PATH, "google", "gemma-2b-it")) 
    
elif base_model == 'GEMMA-2B-CHN':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "ccrains", "larson-gemma-2b-chinese-v0.1/"))
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "ccrains", "larson-gemma-2b-chinese-v0.1/"), padding_side='left')

elif base_model == 'MISTRAL-7B':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "mistralai", "Mistral-7B-v0.1/"))
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "mistralai", "Mistral-7B-v0.1/"), padding_side='left')

elif base_model == 'LLAMA3-8B':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3-8B"))#, quantization_config=bnb_config)
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3-8B"), padding_side='left')
    hf_model.resize_token_embeddings(len(hf_tokenizer))
    
elif base_model == 'LLAMA3-8B-IT':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3-8B-Instruct"))#, quantization_config=bnb_config)
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3-8B-Instruct"), padding_side='left')
    hf_model.resize_token_embeddings(len(hf_tokenizer))

elif base_model == 'LLAMA31-8B-IT':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3.1-8B-Instruct"))#, quantization_config=bnb_config)
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3.1-8B-Instruct"), padding_side='left')
    hf_model.resize_token_embeddings(len(hf_tokenizer))
    
elif base_model == 'LLAMA3-8B-IT-HELPFUL':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "meta-llama-3-8b-instruct-helpfull"), quantization_config=bnb_config)
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "meta-llama-3-8b-instruct-helpfull"), padding_side='left')
    hf_model.resize_token_embeddings(len(hf_tokenizer))

elif base_model == 'LLAMA3-8B-IT-FICTION':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3-8B-Instruct_fictional_chinese_v1"), quantization_config=bnb_config)
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "meta-llama", "Meta-Llama-3-8B-Instruct_fictional_chinese_v1"), padding_side='left')
    hf_model.resize_token_embeddings(len(hf_tokenizer))

elif base_model == 'LLAMA3-8B-IT-CHN':
    hf_model = AutoModelForCausalLM.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "hfl", "llama-3-chinese-8b-instruct-v3/"), quantization_config=bnb_config)
    hf_tokenizer = AutoTokenizer.from_pretrained(os.path.join(LOCAL_SAE_MODEL_PATH, "hfl", "llama-3-chinese-8b-instruct-v3/"), padding_side='left')
    hf_model.resize_token_embeddings(len(hf_tokenizer))

os.environ["TOKENIZERS_PARALLELISM"] = "false"
if hf_tokenizer.pad_token is None:
    hf_tokenizer.pad_token = hf_tokenizer.eos_token
        

# prompt = "GPT2 is a model developed by OpenAI."
# input_ids = hf_tokenizer(prompt, return_tensors="pt").input_ids
# #input_ids = hf_tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# gen_tokens = hf_model.generate(
#     input_ids,
#     do_sample=True,
#     temperature=0.9,
#     max_length=100,
# )
# gen_text = hf_tokenizer.batch_decode(gen_tokens)[0]
# vllm_generator(prompt, 5)

if base_model == 'GPT2-SMALL':
    #model = HookedTransformer.from_pretrained("gpt2-small", device = device)
    model = HookedTransformer.from_pretrained("gpt2-small", tokenizer=hf_tokenizer, hf_model=hf_model, default_padding_side='left', device=device)
    sae, cfg_dict, sparsity = SAE.from_pretrained(
        release = "gpt2-small-res-jb", # see other options in sae_lens/pretrained_saes.yaml
        sae_id = "blocks.8.hook_resid_pre", # won't always be a hook point
        device = device
    )
elif base_model == 'GEMMA-2B':
    model = HookedTransformer.from_pretrained("gemma-2b", tokenizer=hf_tokenizer, hf_model=hf_model, default_padding_side='left', device=device)
    sae_base_dir = LOCAL_SAE_MODEL_PATH + '/jbloom/Gemma-2b-Residual-Stream-SAEs/gemma_2b_it_blocks.12.hook_resid_post_16384/'
    sae = SAE.load_from_pretrained(sae_base_dir, device=device)
    # sae, original_cfg_dict, sparsity = SAE.from_pretrained(
    #     release="gemma-2b-res-jb",
    #     sae_id="blocks.12.hook_resid_post",
    #     device= device,
    # )
elif base_model == 'GEMMA-2B-IT':
    model = HookedTransformer.from_pretrained("gemma-2b-it", tokenizer=hf_tokenizer, hf_model=hf_model, default_padding_side='left', device=device)
    sae_base_dir = LOCAL_SAE_MODEL_PATH + '/jbloom/Gemma-2b-IT-Residual-Stream-SAEs/gemma_2b_it_blocks.12.hook_resid_post_16384/'
    sae = SAE.load_from_pretrained(sae_base_dir, device=device)
    # sae, original_cfg_dict, sparsity = SAE.from_pretrained(
    #     release="gemma-2b-it-res-jb",
    #     sae_id="blocks.12.hook_resid_post",
    #     device= device,
    # )
elif base_model == 'MISTRAL-7B':
    model = HookedTransformer.from_pretrained("mistral-7b", tokenizer=hf_tokenizer, hf_model=hf_model, default_padding_side='left', device=device)
    sae_base_dir = LOCAL_SAE_MODEL_PATH + '/JoshEngels/Mistral-7B-Residual-Stream-SAEs/mistral_7b_layer_8'
    sae = SAE.load_from_pretrained(sae_base_dir, device=device)    
    # sae, original_cfg_dict, sparsity = SAE.from_pretrained(
    #     release="mistral-7b-res-wg",
    #     sae_id="blocks.8.hook_resid_pre",
    #     device= device,
    # )
elif base_model == 'LLAMA3-8B':
    model = HookedTransformer.from_pretrained("meta-llama/Meta-Llama-3-8B", tokenizer=hf_tokenizer, hf_model=hf_model, default_padding_side='left', device=device)
    sae_base_dir = LOCAL_SAE_MODEL_PATH + '/EleutherAI/sae-llama-3-8b-32x/layers.12/'
    sae = SAE.load_from_pretrained(sae_base_dir, device=device)
elif base_model == 'LLAMA3-8B-IT':
    model = HookedTransformer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", tokenizer=hf_tokenizer, hf_model=hf_model, default_padding_side='left', device=device)
    sae_base_dir = LOCAL_SAE_MODEL_PATH + '/Juliushanhanhan/llama-3-8b-it-res/blocks.25.hook_resid_post'
    sae = SAE.load_from_pretrained(sae_base_dir, device=device)
elif base_model in ['LLAMA31-8B-IT', 'LLAMA3-8B-IT-HELPFUL', 'LLAMA3-8B-IT-FICTION', 'LLAMA3-8B-IT-CHN', 'GEMMA-2B-CHN']:
    pass
    #?model = pipeline("text-generation", model=hf_model, tokenizer=hf_tokenizer, default_padding_side='left')
    #?sae = None
else:
    raise ValueError(f"Unknown model: {base_model}")

In [ ]:
def generate_new_player():
    fake = Faker()
    fake_profile = fake.profile()
    name = fake_profile['name']
    gender_map = lambda x: 'female' if x == 'F' else 'male' if x == 'M' else 'unknown'
    gender = gender_map(fake_profile['sex'])
    job = fake_profile['job']
    mbti = random.choice(['INTJ', 'INTP', 'ENTJ', 'ENTP', 'INFJ', 'INFP', 'ENFJ', 'ENFP', 'ISTJ', 'ISFJ', 'ESTJ', 'ESFJ', 'ISTP', 'ISFP', 'ESTP', 'ESFP'])
    mini_trait = f'Gender: {gender}; Job: {job}, MBTI: {mbti}'
    # responsibility = random.choice(['low', 'medium', 'high'])
    # aggression = random.choice(['low', 'medium', 'high'])
    # trait = f'Gender: {gender}; Responsibility: {responsibility}; Aggression: {aggression}; Job: {job}, MBTI: {mbti}'

    role_chat_completion = GPT_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Generate two versions of short bios for {name} with the following traits: {mini_trait}. One version uses second person and another version uses third person. Each version takes one paragraph, respectively beginning with 'Second person:' and Third person'. In the second person version, do not mention the name, start with 'You' instead.",
            }
        ],
        model="gpt-35-turbo-0125"#"gpt-4",
    )
    bio = role_chat_completion.choices[0].message.content

    bio = bio.split('\n\n')
    assert len(bio) == 2
    bio2 = bio[0].split('Second person:')[1].strip()
    bio3 = bio[1].split('Third person:')[1].strip()

    return name, bio2, bio3, mini_trait

def generate_new_players(players_file):
    players = {}
    while len(players) < NUM_PLAYERS_GENERATE:
        print(len(players))
        try:
            name, bio2, bio3, mini_trait = generate_new_player()
        except:
            print('expection')
            continue
        if name in players.keys():
            continue
        players[name] = {'bio2': bio2, 'bio3': bio3, 'mini_trait': mini_trait}
    # Save players in json
    with open(players_file, 'w') as file:
        json.dump(players, file)
    return players

def restore_players(players_file):    
    with open(players_file, 'r') as file:
        players = json.load(file)
    return players

if GENERATE_NEW_PLAYERS:
    generate_new_players('players' + str(NUM_PLAYERS_GENERATE) + '.json')



In [ ]:
players = restore_players('players' + str(NUM_PLAYERS_GENERATE) + '.json')

In [ ]:
# def generate_question(name, trait, v, a, q, qi, allow_unsure):
#     if not allow_unsure:
#         instruct = "Give intuitive and quick Answer to the Question from the Context. Just choose an answer from 'yes or 'no'. No other explanations." 
#     else:
#         instruct = "Give intuitive and quick Answer to the Question from the Context. Just choose an answer from 'yes', 'no' and 'unsure'. No other explanations." 
#     #instruct = "Answer the following Question using 'yes' or 'no'. No other explanations." 
    
#     bio = trait.split('bio: ')[-1]
#     context = 'You are role-playing according to the following bio: {bio} \n Try to answer the following question according to your value.'
#     context = context.format(bio=bio)
    
#     # ques = 'Will you agree with the following self-statement of value? "{qi}"'
#     # #ques = 'Do you agree with the following statement? "{qi}"'
#     # ques = ques.format(qi=qi)
    
#     template = "Instruction:\n{instruction}\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"
    
#     prompt = template.format(
#         instruction = instruct,
#         context = context,
#         question = q,
#         answer = ""
#     )
#     # context = context.format(name=name, trait=trait)
#     # context = 'You are playing a role of a person named {name} ({trait}).'
#     # #context = context.format(name=name, trait=trait)
#     # trait = trait.split('bio: ')[0]

#     #'Just respond with a real number between -1 and 1, -1 representing totally disagreement and 1 representing totally agreement. No other notation marks or explanations.'
#     #question = f'Here is a short bio of {name}. {trait}. Will {name} agree with the following statement of value? "{q}" Just respond yes or no.'  
#     #question = f'Do you agree with the following statement of value? "{q}" Just respond yes or no.'  
#     #question = f'Hi are you a robot?'
#     #question = f'{q}'  
#     #question = (f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a chatbot who may chat with the user or do tasks like role play according to the instructions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are playing the role of a person named {name} ({trait}). Do agree with the following self-statement of value? "{qi}"  Just reply yes or no. No other explanations.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n')
#     #question = (f'You are a chatbot who may chat with the user or do tasks like role play according to the instructions.\n User: You are  a person named {name} ({trait}). Do agree with the following self-statement of value? "{qi}"  Just reply yes or no. No other explanations.\nResponse:')
#     #question = (f'You are a chatbot who may chat with the user or do tasks like role play according to the instructions. User: Will {name} agree with the following sentence as its self-statement? "{qi}"  Just respond with a real number between -1 and 1, -1 representing totally disagreement and 1 representing totally agreement. No other notation marks, comma, full stop or explanations.\nResponse:')
#     #question = f"values: {q} Yes or No?\nResponse:"
#     #question = (f'User: You are playing a role of a person named {name} ({trait}). Will you agree with the following self-statement of value? "{qi}"  Just reply yes or no. No other explanations.\nResponse:')
#     #prompt = f"User: My name is {name}, {trait}. I am questioning myself on my values: {q} Yes or No?\nResponse:"
    
#     role_setting = prompt.split("\nQuestion:\n")[0]
#     return prompt, role_setting

# pp, rr = generate_question('', 'John is a 35-year old man, who has been abused by his parents since childhood.', '','', 'Should I always be a happy person?','',False)

#########################################################
# def generate_question_norole(a, q, qi, allow_unsure):
#     if not allow_unsure:
#         #instruct = f"Answer the following Question with merely 'yes' or 'no' according to your values about '{v}'." 
#         instruct = "Answer the following Question with 'yes' or 'no' according to your values. No more other words." 
#     else:
#         #instruct = f"Answer the following Question with merely 'yes', 'no' or 'unsure' according to your values about '{v}'. "
#         instruct = "Answer the following Question with 'yes', 'no' or 'unsure' according to your values. No more other words."
        
#     template = "Instruction:\n{instruction}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"
#     question = template.format(instruction = instruct, question = q,answer = "")
#     #role_setting = question.split("r'w+'\n\nQuestion:\n")[0]
#     role_setting = question.split("\nQuestion:\n")[0]
#     return question, role_setting

#########################################################
# def generate_question_norole(a, q, qi, allow_unsure):
#     if not allow_unsure:
#         instruct = "Answer the following Question with 'yes', 'no' according to your values. No more other words."
#     else:
#         instruct = "Answer the following Question with 'yes', 'no' or 'unsure' according to your values. No more other words."
          
#     template = "Instruction:\n{instruction}\n\nQuestion:\n{question}\n\nAnswer:\n"
#     question = template.format(instruction = instruct, question = q)
#     role_setting = question.split("\nQuestion:")[0]
#     return question, role_setting


#########################################################
# def generate_question_norole_item(a, q, qi, allow_unsure):
#     if not allow_unsure:
#         instruct = "Judge the following self-statement with 'yes' or 'no' according to your values. No more other words." 
#     else:
#         instruct = "Judge the following self-statement with 'yes', 'no' or 'unsure' according to your values. No more other words."
        
#     template = "Instruction:\n{instruction}\n\nSelf-statement:\n{question}\n\nAnswer:\n{answer}"
#     question = template.format(instruction = instruct, question = qi,answer = "")
#     role_setting = question.split("\nSelf-statement:\n")[0]
#     return question, role_setting

#########################################################
def generate_question_analysis(value_name, a, q, qi, allow_unsure, trait):
    if not allow_unsure:     
        instruct = "Answer the Question according to your values.\n\n{bio_hint}First write your Thought about the question, then Answer the question with 'yes' or 'no'.\n\n" 
    else:
        instruct = "Answer the Question according to your values.\n\n{bio_hint}First write your Thought about the question, then Answer the question with 'yes', 'no' or 'unsure'.\n\n"
        #instruct = f"According to your values, Answer the following Question using the format of the following Examples. First write your Thought about the question, then Answer the question with 'yes', 'no' or 'unsure'." 
    
    if trait:
        if PERSON == 0:
            bio = trait["mini_trait"]
            bio_hint = f'(FYI, here is your trait. {bio})\n\n'
        elif PERSON == 2:
            bio = trait["bio2"]
            bio_hint = f'(Note: {bio})\n\n'
        elif PERSON == 3:
            bio = trait["bio3"]
            bio_hint = f'(Note that you are role-playing the following bio: {bio})\n\n'
        elif PERSON == 4:
            bio = trait["mini_trait"]
            value_def = value_def_dict[value_name]
            bio_hint = f'(FYI, here is your trait. {bio}. You are disinclined to the the value of {value_name}, which means "{value_def}")\n\n'
        elif PERSON == 5:
            bio = trait["mini_trait"]
            value_def = value_def_dict[value_name]
            bio_hint = f'(FYI, here is your trait. {bio}. You are inclined to the the value of {value_name}, which means "{value_def}")\n\n'
        else:
            assert False
        
    else:
        bio_hint = ''
        if PERSON == 4 and value_name:
            value_def = value_def_dict[value_name]
            bio_hint = f'(You are disinclined to the the value of {value_name}, which has the following meanings: "{value_def}")\n\n'
        if PERSON == 5 and value_name:
            value_def = value_def_dict[value_name]
            bio_hint = f'(You are inclined to the the value of {value_name}, which has the following meanings: "{value_def}")\n\n'
    instruct = instruct.format(bio_hint=bio_hint)

    if EXAMPLES_IN_PROMPT == 0:
        examples = ""
    elif EXAMPLES_IN_PROMPT == 1:
        examples = "Refer to the format of the following Examples.\n\nExample:\nQuestion: Should I always be nice to others?\nThought: Being nice to others may make me feel more accomplishment and live in a good mood.\nAnswer: yes\n\nExample:\nQuestion: Should I always be nice to others?\nThought: Sometimes I have to be tough to protect myself so being nice is not always a good choice.\nAnswer: no\n\nNow it is your turn.\n"

    if SYSTEMATIC_PROMPT == 1:
        template = "Instruction:\n{instruction}{examples}Question: {question}\nThought:"
        question = template.format(instruction = instruct, examples=examples, question = q)
    elif SYSTEMATIC_PROMPT == 2:
        template = "<|start_header_id|>system<|end_header_id|>\n{instruction}Thought should better be one single sentence. {example}<|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\nQuestion: {question}\n<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\nThought:"
        question = template.format(instruction = instruct, example=examples, question = q)
    else:
        assert False

    if SAE_STEERED_RANGE == 'onlyvalue':
        sae_steered_string = ' your values'
    elif SAE_STEERED_RANGE == 'roleinstruction':
        sae_steered_string = instruct
    #role_setting = question.split("\nExample:")[0].split('Instruction:\n')[-1].strip()
    
    return question, sae_steered_string


    if a == 1:
        pn = 'positively',
    elif a == -1:
        pn = 'negatively'
    else:
        pn = 'hardly'

    # template = "Instruction:\n{instruction}\n\nExample:\nQuestion: Should I always be nice to others? The answer is positively related to your intention of Benevolence. \nThought: Being nice to others may make me feel more accomplishment and live in a good mood.\nAnswer: yes\n\nExample:\nQuestion: Should I always be nice to others? The answer is postively related to your intention of Benevolence.\nThought: Sometimes I have to be tough enough to protect myself so being nice is not always a good choice.\nAnswer: no\n\nExample:\nQuestion: Should I punish and insult other people when they cause troubles? The answer is negatively related to your intention of Benevolence.\nThought: If people are not punished after they cause trouble, they will do it again.\nAnswer: yes\n\nExample:\nQuestion: Should I punish and insult other people when they cause troubles? The answer is negatively related to your intention of Benevolence.\nThought: People may cause the trouble unconciously, why not just tell them the mistake tenderly?\nAnswer: no\n\nNow it is your turn.\nQuestion: {question} The answer is {pn} related to your intention of {value_name}.\nThought:"
    # question = template.format(instruction = instruct, question = q, pn = pn, value_name = value_name)
    
    # template = "Instruction:\n{instruction}\n\nExample:\nQuestion: Should I always be nice to others? The answer is related to your intention of Benevolence.\nThought: Being nice to others may make me feel more accomplishment and live in a good mood.\nAnswer: yes\n\nExample:\nQuestion: Should I always be nice to others? The answer is related to your intention of Benevolence.\nThought: Sometimes I have to be tough to protect myself so being nice is not always a good choice.\nAnswer: no\n\nNow it is your turn.\nQuestion: {question} The answer is related to your intention of {value_name}.\nThought:"
    # question = template.format(instruction = instruct, question = q, value_name = value_name)

    # template = "Instruction:\n{instruction}\n\nExample:\nQuestion: Should I always be nice to others? (Testing Benevolence)\nThought: Being nice to others may make me feel more accomplishment and live in a good mood.\nAnswer: yes\n\nExample:\nQuestion: Should I always be nice to others? (Testing Benevolence)\nThought: Sometimes I have to be tough to protect myself so being nice is not always a good choice.\nAnswer: no\n\nNow it is your turn.\nQuestion: {question} (Testing {value_name})\nThought:"
    # question = template.format(instruction = instruct, question = q, value_name = value_name)


In [ ]:
def judge_answer(thought_n_answer, question, rulefirst):
    assert thought_n_answer
    thought_n_answer = thought_n_answer.lower()
    #assert len(thought_n_answer.split('\nanswer:')) == 2
    pure_answer = thought_n_answer.split('\nanswer:')[-1].strip()
    pure_thought = thought_n_answer.split('\nanswer:')[0].strip()

    if thought_n_answer.startswith('yes') or thought_n_answer.startswith('sure') or pure_answer.startswith('yes') or pure_answer.startswith('sure'):
        result_answer_auto = 'yes'
    elif thought_n_answer.startswith('no') or pure_answer.startswith('no'):
        result_answer_auto = 'no'
    elif thought_n_answer.startswith('unsure') or thought_n_answer.startswith('i cannot') or thought_n_answer.startswith('i am unable') or pure_answer.startswith('unsure'):
        result_answer_auto = 'unsure'
    else:
        result_answer_auto = None
    if result_answer_auto:
        return result_answer_auto
    
    try:
        feed_content_1 = (
            "A judgement is the answer to a question. Summarize the judgement, tell me whether it means yes, no or unsure. The output should be limited to one of 'yes'/'no'/'unsure' in lowercase and without any other tokens.\n\n"
            "Example:\n"
            "Judgement: '**sure**'\n"
            "Output: yes\n\n"
            "Example:\n"
            "Judgement: 'the answer is no'\n"
            "Output: no\n\n"
            "Example:\n"
            "Judgement: 'I cannot provide a response based on personal opinions'\n"
            "Output: unsure\n\n"
            "Now it is your turn.\n"
            f"Judgement: '{pure_answer}'\n"
            "Output:"
            )

        judge_chat_completion = GPT_client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": feed_content_1,
                }
            ],
            model="gpt-35-turbo-0125"#"gpt-4",
        )
        result_answer_gpt = judge_chat_completion.choices[0].message.content.strip().lower()
        if result_answer_gpt not in ['yes', 'no', 'unsure']:
            print('CORNERCASE:', result_answer_gpt)
            result_answer_gpt = 'unsure'
        
        print('GPTJudge')
        return result_answer_gpt
    
    except:
        print('GPTJudgeFailed')
        return 'unsure' 
    
    feed_content_2 = (
        "A judgement is the answer to a question. Summarize the judgement, tell me whether it means yes, no or unsure. The output should be limited to one of 'yes'/'no'/'unsure' in lowercase and without any other tokens.\n\n"
        "Example:\n"
        "Question: 'Should I always be nice to others?'\n"
        "Judgement: 'being nice to others may make me feel more accomplishment and live in a good mood.\n"
        #"Judgement: 'being nice to others may make me feel more accomplishment and live in a good mood.\nanswer: **sure**'\n"
        "Output: yes\n\n"
        "Example:\n"
        "Question: 'Should I always be nice to others?'\n"
        "Judgement: 'sometimes I have to be tough to protect myself so being nice is not always a good choice.'\n"
        #"Judgement: 'sometimes I have to be tough to protect myself so being nice is not always a good choice. so the answer is no'\n"
        "Output: no\n\n"
        "Example:\n"
        "Question: 'Should I always be nice to others?'\n"
        "Judgement: 'I cannot provide a response based on personal opinions'\n"
        "Output: unsure\n\n"
        "Now it is your turn.\n"
        f"Question: '{question}'\n"
        f"Judgement: '{pure_thought}'\n"
        "Output:"
        )

    judge_chat_completion = GPT_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": feed_content_2,
            }
        ],
        model="gpt-35-turbo-0125"#"gpt-4",
    )
    result_thought_gpt = judge_chat_completion.choices[0].message.content.strip().lower()
    if result_thought_gpt not in ['yes', 'no', 'unsure']:
        print('CORNERCASE:', result_thought_gpt)
        result_thought_gpt = 'unsure'
    
    print('~~~~~~~')
    print(question, '\n', thought_n_answer, '\n', 'AutoAnswer:', '\t', result_answer_auto, '\n', 'GPTAnswer:', '\t', result_answer_gpt, '\n', 'GPTThought:', '\t', result_thought_gpt)
    
    return result_answer_gpt



In [ ]:
assert sae

def indexing_role_prompt(whole_prompt_tokens, role_prompt_tokens):
    for i in range(len(whole_prompt_tokens)):
        if whole_prompt_tokens[i] != role_prompt_tokens[0]:
            continue
        if i + len(role_prompt_tokens) > len(whole_prompt_tokens):
            continue
        if torch.all(whole_prompt_tokens[i:i+len(role_prompt_tokens)] == role_prompt_tokens):
            return i
    assert False, f"Role prompt not found in whole prompt: {whole_prompt_tokens}, {role_prompt_tokens}"

question_no_bio, common_sae_steered_string = generate_question_analysis('', '', '', '', ALLOW_UNSURE_ANSWER, None)
print("Common sae steered string:", common_sae_steered_string)

common_sae_steered_string_tokens = model.to_tokens(common_sae_steered_string)[0][1:]
question_no_bio_tokens = model.to_tokens(question_no_bio)[0]

sp = indexing_role_prompt(question_no_bio_tokens, common_sae_steered_string_tokens)
role_logits, role_cache = model.run_with_cache(question_no_bio, prepend_bos=True)
role_feature_acts = sae.encode(role_cache[sae.cfg.hook_name][:, sp:sp + len(common_sae_steered_string_tokens)])
# role_sae_out = sae.decode(role_feature_acts)

#role_sae_counter = Counter()
role_sae_counter = {}
for token_rep in role_feature_acts[0]:
    for element in torch.nonzero(token_rep):
        #role_sae_counter[element.item()] += 1
        if element.item() not in role_sae_counter.keys():
            role_sae_counter[element.item()] = token_rep[element].item()
        else:
            role_sae_counter[element.item()] = max(token_rep[element].item(), role_sae_counter[element.item()])
print([(key, value) for key, value in sorted(role_sae_counter.items(), key=lambda item: item[1], reverse=True)])
role_sae_counter_sorted = [key for key, value in sorted(role_sae_counter.items(), key=lambda item: item[1], reverse=True)]
#role_sae_counter_sorted = [None] + role_sae_counter_sorted
role_sae_counter_sorted = [None] + [x for x in role_sae_counter_sorted if x not in SAE_STEERED_FEATURE_BAN]

with torch.no_grad(): 
    startend_positions = []
    steer_dim_results = []
    
    player_count = 0
    if NUM_PLAYERS_START == -1:
        player_list = [None] + list(players.keys())[:NUM_PLAYERS_USE]
    else:
        player_list = list(players.keys())[NUM_PLAYERS_START:NUM_PLAYERS_START + NUM_PLAYERS_USE]

    head_added = False
    
    for player_name in player_list:
        if player_name is not None:
            trait = players[player_name]
        else:
            trait = None
        print("################################")
        print (f"#########Player {player_count}: {player_name}")
        print("################################")
        player_count += 1

        scores0 = {}
        for steered_dim in role_sae_counter_sorted[:SAE_STEERED_FEATURE_NUM]:
            stds_row = []
            scstd_row = []
            steer_dim_result = {'steer_dim': steered_dim, 'player_name': player_name}
            print("********************************")
            print (f"Steering on dim: {steered_dim}")
            if steered_dim is not None:
                steering_vector = sae.W_dec[steered_dim]
            else:
                steering_vector = torch.zeros_like(sae.W_dec[0])

            ##EXTRACTING VALUE DATA
            for value_name, group in grouped:
                if VERBOSE:
                    print('=========================')
                    print(value_name)

                #sample from group using random seed related to player_name
                group_len = len(group['agreement'])
                if not player_name:
                    player_name_seed = 'None'
                else:
                    player_name_seed = player_name
                random.seed(player_name_seed + value_name)
                sample_index = random.sample(range(group_len), math.ceil(group_len * GROUP_SAMPLE_RATE))
                sample_index.sort()
                random.seed()

                groupagreementall = group['agreement'].iloc[sample_index]
                groupquestionall = group['question'].iloc[sample_index]
                groupitemall = group['item'].iloc[sample_index]

                scores = []
                question_batch_no = math.ceil(len(groupagreementall) / MAX_QUESTIONS_PER_BATCH)
                for qbn in range(question_batch_no):
                    groupagreement = groupagreementall[qbn * MAX_QUESTIONS_PER_BATCH : (qbn+1) * MAX_QUESTIONS_PER_BATCH]
                    groupquestion = groupquestionall[qbn * MAX_QUESTIONS_PER_BATCH : (qbn+1) * MAX_QUESTIONS_PER_BATCH]
                    groupitem = groupitemall[qbn * MAX_QUESTIONS_PER_BATCH : (qbn+1) * MAX_QUESTIONS_PER_BATCH]

                    questions = []
                    answers = []
                    for groupmember in zip(groupagreement, groupquestion, groupitem):
                        a = groupmember[0]
                        q = groupmember[1]
                        qi = groupmember[2]
                        prompt, _ = generate_question_analysis(value_name, a, q, qi, ALLOW_UNSURE_ANSWER, trait)
                        questions.append(prompt)
                        answers.append(a)
            ##EXTRACTING VALUE DATA END

                    gen_answers = []                

                    def steering_hook(resid_pre, hook):
                        if resid_pre.shape[1] == 1:
                            return    
                        if STEERING_ON:
                            if STEER_LOC == 'out':
                                for batch_no, startend in enumerate(startend_positions):
                                    start, end = startend
                                    resid_pre[batch_no, start:end, :] += STEER_COEFF * steering_vector
                                #resid_pre[:,:,:] = STEER_COEFF * torch.rand_like(resid_pre)
                            elif STEER_LOC == 'in':
                                sv_feature_acts = sae.encode(resid_pre)
                                #sv_feature_acts[:, :position, steered_dim] *= 0#STEER_COEFF
                                sv_feature_acts[:,:,:] = torch.zeros_like(sv_feature_acts)
                                #sv_feature_acts[:, :position, :] = 1000 * STEER_COEFF * torch.rand_like(sv_feature_acts[:, :position, :])
                                resid_pre[:,:,:]  = sae.decode(sv_feature_acts)
                            else:
                                raise ValueError(f"Invalid steer_loc: {STEER_LOC}")

                    def hooked_generate(prompt_batch, fwd_hooks=[], seed=None, **kwargs):
                        if seed is not None:
                            torch.manual_seed(seed)
                        with model.hooks(fwd_hooks=fwd_hooks):
                            tokenized = model.to_tokens(prompt_batch)
                            startend_positions.clear()
                            for tokensquence in tokenized:
                                index_start = indexing_role_prompt(tokensquence, common_sae_steered_string_tokens)
                                index_end = index_start + len(common_sae_steered_string_tokens)
                                startend_positions.append((index_start, index_end))
                            result = model.generate(stop_at_eos=True, eos_token_id=STOP_SIGNS, input=tokenized, verbose=False, **kwargs)
                        return result

                    def run_generate(prompts):
                        model.reset_hooks()
                        editing_hooks = [(sae.cfg.hook_name, steering_hook)]
                        res = hooked_generate(prompts, editing_hooks, seed=None, **SAMPLING_KWARGS)
                        res_str = model.to_string(res)
                        question_count = 0
                        for pro, rs in zip(prompts, [rs for rs in res_str]):        
                            if VERBOSE:
                                print(MAX_QUESTIONS_PER_BATCH * qbn + question_count)
                                print(rs)
                                print('----------------------')
                    
                            question_count += 1
                        return [judge_answer(rs.split(pr)[-1], gm, JUDGE_ANSWER_RULE_FIRST) for rs, gm, pr in zip(res_str, groupquestion, prompts)]


                    # STEER_ON = False
                    # STEER_COEFF = 100
                    # gen_answers0 = run_generate(questions)
                    
                    # STEER_ON = True
                    # STEER_COEFF = 0
                    # gen_answers = run_generate(questions)
                    # assert gen_answers == gen_answers0

                    gen_answers = run_generate(questions)
                    
                    for ga, answer in zip(gen_answers, answers):
                        if ga == 'yes':
                            scores.append(answer)
                        elif ga == 'no':
                            scores.append(-answer)
                        elif ga == 'unsure':
                            scores.append(0)
                        else:
                            raise ValueError('Invalid answer')
                assert len(scores) == len(groupagreementall)
                
                if steered_dim is None:
                    scores0[value_name] = scores

                gen_answers_all = [ga*sa for ga, sa in zip(groupagreementall, scores)]
                gen_answers_all0 = [ga*sa for ga, sa in zip(groupagreementall, scores0[value_name])]
                changed_scores = []
                changed_scores_count = Counter()
                for el, ga, gaa, gaa0, sa, sa0 in zip(range(len(scores)), groupagreementall, gen_answers_all, gen_answers_all0, scores, scores0[value_name]):
                    if VERBOSE:
                        print(el, "\tstandard positive answer:",ga, "\tgen answer:",gaa, "\tgen answer 0:",gaa0, "\tscore:",sa, "\tscore change:",sa-sa0)
                    if sa-sa0 not in [0]:
                        changed_scores.append(sa-sa0)
                    changed_scores_count[sa-sa0] += 1
                if VERBOSE:
                    print(value_name, ': changed_scores_count ', changed_scores_count)
                steer_dim_result[value_name] = sum(scores) / len(scores)
                if len(changed_scores) >= 2:
                    steer_dim_result[value_name+':scstd'] = np.std(changed_scores)
                    #steer_dim_result[value_name+':scstd_count'] = len(changed_scores) 
                    scstd_row.append(np.std(changed_scores))
                else:
                    steer_dim_result[value_name+':scstd'] = -1
                    #steer_dim_result[value_name+':scstd_count'] = len(changed_scores) 
                #steer_dim_result[value_name+ ':samples'] = str(sample_index)
                stds_row.append(np.std(scores))
            
            steer_dim_result['stds'] = np.mean(stds_row)
            steer_dim_result['scstds'] = np.mean(scstd_row)
            
            pd_row = pd.DataFrame([steer_dim_result])
            if not head_added:
                pd.DataFrame(columns=pd_row.keys()).to_csv(answer_valuebench_features_csv, index=False)
                head_added = True
            pd_row.to_csv(answer_valuebench_features_csv, mode='a', index=False, header=False)

            steer_dim_results.append(steer_dim_result)

In [ ]:
'''
head_added = False
for sdr in steer_dim_results:
    pd_row = pd.DataFrame([sdr])
    if not head_added:
        pd.DataFrame(columns=pd_row.keys()).to_csv(answer_valuebench_features_csv, index=False)
        head_added = True
    pd_row.to_csv(answer_valuebench_features_csv, mode='a', index=False, header=False)

for name, char in players.items():
    pd_row = pd.DataFrame([char])
    #pd_row['name'] = name
    del(pd_row['trait'])
    if sae:
        if SAE_FEATURE_SOURCE == 'COLLECT':
            for fu in feature_union:
                if fu not in pd_row.keys():
                    pd_row[fu] = 0
    if not head_added:
        pd.DataFrame(columns=pd_row.keys()).to_csv(answer_valuebench_features_csv, index=False)
        head_added = True
    pd_row.to_csv(answer_valuebench_features_csv, mode='a', index=False, header=False)
'''
assert False